In [22]:
import pandas as pd
import dask
from glob import glob
from fbd import get_permno_returns
import time

In [2]:
dask.config.set(scheduler='processes')

# Data

In [3]:
const_mat = pd.read_csv('data/russell3000.csv.gz', compression='gzip',
                        index_col=0)

In [4]:
const_mat.head()

,10026,10032,10051,10065,10104,10107,10137,10138,10143,10145,...,93419,93420,93422,93423,93427,93428,93429,93431,93433,93436
2010-01,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-02,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-03,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-04,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-05,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
info = pd.read_csv('data/permno_info.csv.gz', compression='gzip',
                   index_col=0)
info['permno'] = info['permno'].astype(int)

In [6]:
info.shape

(16336, 4)

In [7]:
info.head()

,permno,comnam,naics,ticker
0,10026,J & J SNACK FOODS CORP,NaN,JJSF
1,10026,J & J SNACK FOODS CORP,311821.0,JJSF
2,10026,J & J SNACK FOODS CORP,311813.0,JJSF
3,10032,PLEXUS CORP,NaN,PLXS
4,10032,PLEXUS CORP,334412.0,PLXS


In [8]:
info.isna().sum()

permno       0
comnam       0
naics     5496
ticker     435
dtype: int64

In [24]:
%%time
res = get_permno_returns('10026', const_mat)

CPU times: user 234 ms, sys: 38.5 ms, total: 272 ms
Wall time: 1.57 s
